<a href="https://colab.research.google.com/github/pmontman/tmp_choicemodels/blob/main/nb/tutorials/WK_10_tuto_efficient_designs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to design efficient experiments

* We will create a fake design matrix for the full experiment
* We will use biogeme to compute the initial choice probabilities
* Compute the d-efficiency for the design
* Show can we can get subsets of the full design and compute their efficiency
* Do a random search for the best subdesign originating from the initial design

In [1]:
!pip install biogeme

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.expressions as exp
import biogeme.tools as tools

In [3]:
betas = np.matrix(' 1 1; 2 2')

In [4]:
def choice_prob(betas, X):
  V = np.matmul(X, betas)
  P = np.exp(V)
  return P / np.sum(P, axis = 1)

In [5]:
colnames = ['price_apple', 'size_apple', 'os_apple', 'price_android', 'size_android', 'os_android']

# Working example, discrete choice experiment for automobile preferences.

We want to understand population preferences for cars, we will consider the following variables.

`price`, `power`, `engine_type`

Ignoring all realistic values, let's go for:

* Prices: 20000, 30000, 40000 AUD.
* Consider power: 130 hp, 170 hp, 220 hp.
* Engine types, encoded as integer initially: 0=petrol, 1=diesel, 2=hybrid, 3=electric.



#Creating all combinations of variables and values

We can create the full factorial design in python by using the cartesian product
`cartesian` function.

Here is an example of use.

In [6]:
from sklearn.utils.extmath import cartesian

full_fact = pd.DataFrame(cartesian(([20000.0, 30000.0, 40000.0], [130, 170, 220], [0, 1, 2, 3, 4])), columns=['price', 'power', 'engine_type'])
full_fact

,price,power,engine_type
0,20000.0,130.0,0.0
1,20000.0,130.0,1.0
2,20000.0,130.0,2.0
3,20000.0,130.0,3.0
4,20000.0,130.0,4.0
5,20000.0,170.0,0.0
6,20000.0,170.0,1.0
7,20000.0,170.0,2.0
8,20000.0,170.0,3.0
9,20000.0,170.0,4.0


These are all possible combinations of price, car and engine type.

In practice the full factorial could be too large to compute. We do not need to
actually compute it *completely* for what we want to do, which is finding the best subset of size $N$ out of the full factorial.

We still need to create the alternatives to compare, imagine that we have two car manufacturers, each will identify one alternative. The two alternatives are then Toyota and Renault. Comparing all possible Toyota values for the attributes vs all possible Renault attributes will render 2025 entries in the full factorial.

Let us create the full experiment comparing two alternatives, we can do that by using the `cartesian` product again, or in this case we `merge` together two copies of the attributes that we computed earlier. `merge` takes two dataframe and a pair of `suffixes` to identify the repeated names of the columns.

In [7]:
design_mat = pd.merge(full_fact, full_fact, how='cross', suffixes=('_toyo', '_rena'))
design_mat

,price_toyo,power_toyo,engine_type_toyo,price_rena,power_rena,engine_type_rena
0,20000.0,130.0,0.0,20000.0,130.0,0.0
1,20000.0,130.0,0.0,20000.0,130.0,1.0
2,20000.0,130.0,0.0,20000.0,130.0,2.0
3,20000.0,130.0,0.0,20000.0,130.0,3.0
4,20000.0,130.0,0.0,20000.0,130.0,4.0
...,...,...,...,...,...,...
2020,40000.0,220.0,4.0,40000.0,220.0,0.0
2021,40000.0,220.0,4.0,40000.0,220.0,1.0
2022,40000.0,220.0,4.0,40000.0,220.0,2.0
2023,40000.0,220.0,4.0,40000.0,220.0,3.0


#Efficiency

Recall that we want to find the subset of $N$ rows from the full experiment that maximizes the efficiency of the resulting experiment.
There are several concepts of efficiency, relatively similar but we will
focus on D-efficiency. Roughly speaking, D-efficiency want to make the covariance matrix for the coefficients, $\text{covariance}(B)$, as 'small' as possible.
In discrete choice, the formula for the covariance matrix of the coefficients is a bit more complex than for linear regression.


$$\text{covariance}(\beta) = (Z' P Z )^{-1}$$

when working with $J$ alternatives:
*  $P$ is the matrix of choice probabilities computed by the model.
* $Z$ is similar to design matrix, but 'centered' using the choice probabilities. Basically, to each row of observations, we substract the weighted mean of the variables across all alternatives. The weights are the choice probabilities computed by the model.

 $$z_{jn} = x_{jn} - \sum_{i=1}^Jx_{in}P_{in}$$

To compute the $Z$ matrix, **we need some  'choice probabilities'**. In our context, we do not yet know these choice probabilities, so we need to work with an initial guess of them. This initial guess usually comes from an 'initial' value for the coefficients that creates equal choice probs, basically a 'no-information' stating model. In some cases (as in the group assignment!), we might get a good starting guess, fom example, if we have data of a similar problem or from a similar experiment.

#Creating the centered design matrix $Z$

We need some initial model that we can use to compute choice probabilities.
We can use a biogeme model, or some manual computation, since we do not really need to estimate the coefficients from the data.

Lets try biogeme.

Load some auxiliary functions first:

---
---

# Auxiliary functions

The first function takes the dictionary of utilities, a pandas dataframe, and the name of the variable that contains the variable with the results of the choice. It returns the biogeme object with the model and the estimated 'results' object (the one we get the values, likelihoods, etc.)
We have added the dictionary with the utilities to the biogeme object, in case we use it later.

In [8]:
def qbus_estimate_bgm(V, pd_df, tgtvar_name, modelname='bgmdef'):
 av = {1: 1,
       2: 1}
 bgm_db = db.Database(modelname + '_db', pd_df)
 globals().update(bgm_db.variables)
 logprob = models.loglogit (V , av , bgm_db.variables[tgtvar_name] )
 bgm_model = bio.BIOGEME ( bgm_db, logprob )
 bgm_model.utility_dic = V.copy()
 return bgm_model, bgm_model.estimate()

The next function will calculate the predictions for a given biogeme object that was estimated with `qbus_estimate_bgm`. The output is the array with the choice probabilities. From the choice probabilities, this can be used to calculate accuracies, confusion matrices and the output of what-if scenarios.

In [9]:
def qbus_simulate_bgm(qbus_bgm_model, betas, pred_pd_df):
  av_auto = qbus_bgm_model.utility_dic.copy()
  for key, value in av_auto.items():
   av_auto[key] = 1

  targets = qbus_bgm_model.utility_dic.copy()
  for key, value in targets.items():
   targets[key] = models.logit(qbus_bgm_model.utility_dic, av_auto, key)

  bgm_db = db.Database('simul', pred_pd_df)
  globals().update(bgm_db.variables)
  bgm_pred_model = bio.BIOGEME(bgm_db, targets)
  simulatedValues = bgm_pred_model.simulate(betas)
  return simulatedValues

The function `qbus_calc_accu_confusion` calculates the accuracies given the choice probability predictions a pandas dataset and the specification of the name that contains the actual choices in the input dataset.

In [10]:
def qbus_calc_accu_confusion(sim_probs, pd_df, choice_var):
  which_max = sim_probs.idxmax(axis=1)
  data = {'y_Actual':   pd_df[choice_var],
          'y_Predicted': which_max
        }

  df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
  confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
  accu = np.mean(which_max == pd_df[choice_var])
  return accu, confusion_matrix 

The next function calculates the likelihood ratio test having to write a bit less code that the default biogeme function. The arguments are the results objects of the two models to be compared. The first is the more complex and the second is the reference model (**the order is important!**). The third argument is the significance level for the test.

In [11]:
def qbus_likeli_ratio_test_bgm(results_complex, results_reference, signif_level):
  return tools.likelihood_ratio_test( (results_complex.data.logLike, results_complex.data.nparam),
                                     (results_reference.data.logLike, results_reference.data.nparam), signif_level)

The next function just updates the globals so we can use it 

In [12]:
def qbus_update_globals_bgm(pd_df):
   globals().update(db.Database('tmp_bg_bgm_for_glob', pd_df).variables)

---
---

#'Estimating' the initial biogeme model

We will create a dataset that is the same as the design matrix, to pass to biogeme. We will get a biogeme model with the right structure and use biogeme for the predictions.
We have to give biogeme a 'fake' response variable that does not exist so it can create the model, otherwise it will give us an error.

We can give arbitrarily values to the choice variable, since we are going to impose the coefficients manually afterwards. We set all choices to 1, then, we need to get some of the entries to the other alternative (otherwise biogeme will go crazy), so we set the first row to 2.

In [13]:
init_dset = design_mat.copy()
init_dset['choice'] = 1
qbus_update_globals_bgm(init_dset)

In [14]:
init_dset['choice'][0] = 2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Now we create the biogeme model that would analyze the data, as usual.
The differences is that, for simplicity, we will set most coefficients to 0 and tell biogeme to **not fit them.** This is because we would like to set them manually to 0 so we have the no-information starting coefficients. If we have other dataset that could be used for the initial guess, then we would fit it for real.

Unfortunately, we cannot set all coefficients to 0 at this stage, we have to fit some coefficient (otherwise biogeme will give an error) so we fill fit one of the ASC (we will ignore the result anyway).

In [15]:
ASC_toyo = exp.Beta ( 'ASC_toyo' ,0, None , None ,1)
ASC_rena = exp.Beta ( 'ASC_rena' ,0, None , None ,0)

B_price = exp.Beta ( 'B_price',0, None , None ,1)
B_power = exp.Beta ( 'B_power',0, None , None ,1)
B_engine_type = exp.Beta('B_engine_type', 0, None, None, 1)

In [16]:
V_toyo = ASC_toyo + B_price*price_toyo + B_power*power_toyo + B_engine_type*engine_type_toyo
V_rena = ASC_rena + B_price*price_rena + B_power*power_rena + B_engine_type*engine_type_rena

V_base = {1: V_toyo,
     2: V_rena}

And we now 'estimate' the biogeme model, remember this estiamtion is so we can get the structure for making the predictions, but we are going to ignore the actual estimated coefficients and set them to 0.

In [17]:
model_base, results_base = qbus_estimate_bgm(V_base, init_dset, 'choice', 'automob')

[12:20:43] < Warning >   You have not defined a name for the model. The output files are named from the model name. The default is [biogemeModelDefaultName]


In [29]:
results_base.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_rena,-7.612799,1.000215,-7.611162,2.708944e-14


**Important step**. We need to compute the choice probabilities of the initial model. We will use biogeme for that, but where we pass the Betas, we will set them manually. Notice the dictionary `{'ACS_rena':0}`, this will tell biogeme that the value of the ASC that we fitted is set to 0, manually by us. The rest of the betas are not needed to set manually to 0, because they were set fixed to zero at the time of the definition.

We calculate the choice probabilties for the design matrix, we should get equal probs. because the Betas are set to 0, so all alternatives haver the same utility for all values of their attributes.

In [18]:
init_choice_probs=qbus_simulate_bgm(model_base, {'ASC_rena':0}, design_mat)
init_choice_probs

,1,2
0,0.5,0.5
1,0.5,0.5
2,0.5,0.5
3,0.5,0.5
4,0.5,0.5
...,...,...
2020,0.5,0.5
2021,0.5,0.5
2022,0.5,0.5
2023,0.5,0.5


# Estimating the d-efficiency for choice models.

With the initial choice probabilities and the design matrix we can compute the efficiency following the above for the covariance for the betas, and the formula in the lectures for the d-efficienty based on the determinant of the covariance of the betas.

Programmatically it is a bit tricky, so we have this function `calc_mnl_cov`. 

 **Important, you might want to reuse it for the group assignment**. The function gets as arguments, a design matrix, the choice probabilities, and we have also to pass them the number of alternatives and the number of attribures per alternatives.

In [19]:
def calc_mnl_cov(design_m, cprobs, num_alt, attrs_per_alt):
  P_rep = np.repeat(cprobs.to_numpy(), np.repeat(attrs_per_alt, num_alt), axis=1)
  XP_rep = np.repeat((design_m.to_numpy()*P_rep).sum(axis=1).T.reshape(-1,1), 6, axis=1)
  Z = design_m - XP_rep
  ZPZ = np.matmul(Z.T, P_rep*Z.to_numpy())
  covMNL = np.linalg.inv(ZPZ)
  return covMNL

In [20]:
covMNL = calc_mnl_cov(design_mat, init_choice_probs, 2, 3)

The definition of d-efficiency:

In [21]:
def d_effic(covMAT):
  return np.power( np.linalg.det(covMAT), 1 / (covMAT.shape[0] + 1) )

And we can calculate the efficiency, finally.

In [22]:
d_effic(covMNL)

7.123314013365285e-07

#Finding good designs by random search

Our actual goal is to find a good subset of the full factorial design matrix.
There are some sophisticated ways of findining it, it is an optimization problem that mathematicians have studied.

We will use random search, we will compare many possible subsets of the full design matrix, and pick the best one.




Set the size of the experiment, say 100 rows.

In [23]:
N = 100

Get a random subset of the design matrix.
The python function `np.random.choice` generates random integers for a given range. We pass as range, the number of rows in the full design matrix.
We use sampling without replacement, but you can use replacement, it could repeat rows of the design, but as long as the efficiency is OK, we are good to go.

In [24]:
np.random.seed(1234) 

selected_rows = np.random.choice(design_mat.shape[0], N, replace=False)
selected_rows

array([ 685,  878,  486,  937,  891,  362,  456, 1878,  984,  894, 1741,
        881,  995,   69,  697, 1523,  310,  184, 1381, 1928,   90, 1303,
       1468,  265,  466, 1418,  114,   21,  123,  483, 1190,  802, 1933,
        202,  218,  492, 1476,  808,  266,  338,  463, 1446, 1910, 1670,
       1635, 1157,  316,  276, 1644, 1023, 1888, 1113,  220,  234,  836,
       1563,  284, 1456,  570,  791, 1719,  216, 1861, 1375, 1049,  709,
        884, 1602, 1823, 1409,  950,  214, 1988,  839, 1419,  315, 1143,
       1651, 1742,  646, 1086, 1291,  188, 1199, 1334,  441, 1637,  668,
        921, 1107,  319,  190,  982,  154, 1634, 1646,  750, 1198, 1522,
       1779])

Lets look at how this subdesign looks like

In [25]:
sub_design = design_mat.iloc[selected_rows,:]
sub_design

,price_toyo,power_toyo,engine_type_toyo,price_rena,power_rena,engine_type_rena
685,30000.0,130.0,0.0,20000.0,220.0,0.0
878,30000.0,130.0,4.0,30000.0,170.0,3.0
486,20000.0,220.0,0.0,40000.0,170.0,1.0
937,30000.0,170.0,0.0,40000.0,170.0,2.0
891,30000.0,130.0,4.0,40000.0,170.0,1.0
...,...,...,...,...,...,...
1646,40000.0,170.0,1.0,30000.0,220.0,1.0
750,30000.0,130.0,1.0,40000.0,130.0,0.0
1198,30000.0,220.0,1.0,30000.0,220.0,3.0
1522,40000.0,130.0,3.0,40000.0,170.0,2.0


Compute the efficiency for that design.

In [26]:
sub_probs = qbus_simulate_bgm(model_base, {'ASC_rena':0}, sub_design)
sub_covMNL = calc_mnl_cov(sub_design, sub_probs, 2, 3)
d_effic(sub_covMNL)

9.69990271225827e-06

Put everything in a `for` loop, and run 250 times, pick the best sub design.

In [27]:
best_rows = None
best_effic = 9999
for i in range(250):
  selected_rows = np.random.choice(design_mat.shape[0], N, replace=False)
  sub_design = design_mat.iloc[selected_rows,:]
  sub_probs = qbus_simulate_bgm(model_base, {'ASC_rena':0}, sub_design)
  sub_covMNL = calc_mnl_cov(sub_design, sub_probs, 2, 3)
  d_ef = d_effic(sub_covMNL)
  if (d_ef < best_effic):
    best_effic = d_ef
    print('New best efficiency found!', best_effic)
    best_rows = selected_rows


New best efficiency found! 9.403906999879586e-06
New best efficiency found! 9.356175086759439e-06
New best efficiency found! 9.08633026111215e-06
New best efficiency found! 9.001191933946822e-06
New best efficiency found! 8.966224621314157e-06


Lets print the best design found

In [28]:
design_mat.iloc[best_rows,:]

,price_toyo,power_toyo,engine_type_toyo,price_rena,power_rena,engine_type_rena
1544,40000.0,130.0,4.0,20000.0,220.0,4.0
34,20000.0,130.0,0.0,40000.0,130.0,4.0
415,20000.0,170.0,4.0,20000.0,220.0,0.0
1173,30000.0,220.0,1.0,20000.0,130.0,3.0
1483,40000.0,130.0,2.0,40000.0,220.0,3.0
...,...,...,...,...,...,...
1400,40000.0,130.0,1.0,20000.0,170.0,0.0
1160,30000.0,220.0,0.0,40000.0,170.0,0.0
133,20000.0,130.0,2.0,40000.0,220.0,3.0
1313,30000.0,220.0,4.0,20000.0,170.0,3.0


#Exercise 1) Think of some good initial coefficients that are not zero, and run the search again using that model for the choice probabilities. Notice the differences? What are the choice probs of the original subset found (under the new coefficients)  vs the new subset found?

#Exercise 2) Encode the engine_type as binary dummy and find a good experiment of 100 rows.